## 1. 数据读取

In [1]:
import pandas as pd
import json
df = pd.read_csv('house_info.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79795 entries, 0 to 79794
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   title                  79795 non-null  object 
 1   follower_numbers       79795 non-null  int64  
 2   total_price            79795 non-null  float64
 3   unit_price             79795 non-null  int64  
 4   build_time             79795 non-null  object 
 5   region                 79795 non-null  object 
 6   house_type             79795 non-null  object 
 7   floor                  79795 non-null  object 
 8   house_area             79795 non-null  object 
 9   house_type_structure   79795 non-null  object 
 10  inside_area            79795 non-null  object 
 11  building_type          79795 non-null  object 
 12  orientation            79795 non-null  object 
 13  building_structure     79795 non-null  object 
 14  decoration             79795 non-null  object 
 15  us

## 2. 数据预处理

In [2]:
# 删除包含缺失值的列
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79795 entries, 0 to 79794
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   title                  79795 non-null  object 
 1   follower_numbers       79795 non-null  int64  
 2   total_price            79795 non-null  float64
 3   unit_price             79795 non-null  int64  
 4   build_time             79795 non-null  object 
 5   region                 79795 non-null  object 
 6   house_type             79795 non-null  object 
 7   floor                  79795 non-null  object 
 8   house_area             79795 non-null  object 
 9   house_type_structure   79795 non-null  object 
 10  inside_area            79795 non-null  object 
 11  building_type          79795 non-null  object 
 12  orientation            79795 non-null  object 
 13  building_structure     79795 non-null  object 
 14  decoration             79795 non-null  object 
 15  us

In [10]:
# 提取floor楼层中的数字
df['floor'] = df['floor'].str.extract(r'(\d+)', expand=False).astype('int')
# 将房价面积由“85.99m²”-->“85.99”
df['house_area'] = df['house_area'].apply(lambda x: x[:-1]).astype('float')
# 将region列中值后添加“区”，如“北碚”-->“北碚区”
df['region'] = df['region'] + '区'

In [3]:
# 数量、均值、标准差、最小值、四分之一分位数、二分之一分位数、四分之三分位数、最大值
df.describe()

follower_numbers   total_price    unit_price
count      79795.000000  79795.000000  79795.000000
mean           3.789598    134.186112  12650.396416
std           14.099330     91.983921   4940.715010
min            0.000000      5.800000    846.000000
25%            0.000000     78.500000   9500.000000
50%            0.000000    115.000000  12260.000000
75%            2.000000    165.000000  15369.000000
max          518.000000   2880.000000  59209.000000

## 3. 重庆各区二手房数量条形图

In [5]:
import pyecharts.options as opts
from pyecharts.charts import Bar
from pyecharts.globals import ThemeType

region_list = df['region'].value_counts().index.tolist()
house_count_list = df['region'].value_counts().values.tolist()

c = Bar(init_opts=opts.InitOpts(theme=ThemeType.PURPLE_PASSION))
c.add_xaxis(region_list)
c.add_yaxis("重庆市", house_count_list)
c.set_global_opts(title_opts=opts.TitleOpts(title="重庆各区二手房数量柱状图", subtitle=""),
                                     xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(interval=0)))
c.render("./html/重庆各区二手房数量条形图.html")
c.render_notebook()

## 4. 重庆市各区房价分布2D地图

In [6]:
import json
region_list = df['region'].value_counts().index.tolist()
median_unit_price = []
for region in region_list:
    median_unit_price.append(df.loc[df['region'] == region, 'unit_price'].median())

    
# 绘制2D地图
from pyecharts.charts import Map
# 加载武汉市地图数据
json_data = json.load(open('重庆市.json', encoding='utf-8'))

data_pair = [list(z) for z in zip(region_list, median_unit_price)]
    
text_style = opts.TextStyleOpts(color='#fff')
c = Map(init_opts=opts.InitOpts(width='1500px', height='700px', bg_color='#E5D5E8'))    
c.add_js_funcs("echarts.registerMap('重庆市',{});".format(json_data))
c.add(series_name="重庆市", data_pair=data_pair, maptype="重庆市", label_opts=opts.LabelOpts(color='#fff'))
c.set_global_opts(legend_opts=opts.LegendOpts(textstyle_opts=text_style), 
                  title_opts=opts.TitleOpts(title="重庆", title_textstyle_opts=text_style)
                  ,visualmap_opts=opts.VisualMapOpts(split_number=6, max_=30000, range_text=['高', '低'], 
                                                     textstyle_opts=text_style))
c.render("./html/重庆市各区房价分布2D地图.html")
c.render_notebook()

## 5. 重庆市各区房价分布3D地图.html

In [7]:
from pyecharts.charts import Map3D
from pyecharts.globals import ChartType
from pyecharts.commons.utils import JsCode

json_region = []
json_center = []
for feature in json_data['features']:
    json_region.append(feature['properties']['name'])
    json_center.append(feature['properties']['center'])
data_pair = []

for region, center in zip(json_region, json_center):
    median_price = df.loc[df['region'] == region, 'unit_price'].median()
    data_pair.append((region, center + [median_price]))

    
# 绘制3D地图
c = Map3D(init_opts=opts.InitOpts(width='1500px', height='600px'))
    
c.add_js_funcs("echarts.registerMap('重庆市',{});".format(json_data))
c.add_schema(
    maptype='重庆市',
    itemstyle_opts=opts.ItemStyleOpts(
        color="rgb(5,101,123)",
        opacity=1,
        border_width=0.8,
        border_color="rgb(62,215,213)",
    ),
    map3d_label=opts.Map3DLabelOpts(
        is_show=False,
        formatter=JsCode("function(data){return data.name + " " + data.value[2];}"),
    ),
    emphasis_label_opts=opts.LabelOpts(
        is_show=False,
        color="#fff",
        font_size=10,
        background_color="rgba(0,23,11,0)",
    ),
    light_opts=opts.Map3DLightOpts(
        main_color="#7CA9EE",
        main_intensity=1.2,
        main_shadow_quality="high",
        is_main_shadow=False,
        main_beta=10,
        ambient_intensity=0.3,
    ),
)
c.add(
    series_name="重庆市",
    data_pair=data_pair,
    type_=ChartType.BAR3D,
    bar_size=1,
    shading="lambert",
    label_opts=opts.LabelOpts(
        is_show=False,
        formatter=JsCode("function(data){return data.name + ' ' + data.value[2];}"),
    )

)
c.set_global_opts(title_opts=opts.TitleOpts(title="重庆市各区房价分布3D地图.html"))

c.render("./html/重庆市各区房价分布3D地图.html")
c.render_notebook()

## 6. 重庆各区二手房单价箱型图

In [8]:
# 统计各个区二手房单价信息
unit_price_list = []
for region in region_list:
    unit_price_list.append(df.loc[df['region'] == region, 'unit_price'].values.tolist())


# 绘制箱型图
from pyecharts.charts import Boxplot

c = Boxplot(init_opts=opts.InitOpts(theme=ThemeType.PURPLE_PASSION))
c.add_xaxis(region_list)
c.add_yaxis("重庆市", c.prepare_data(unit_price_list))
c.set_global_opts(title_opts=opts.TitleOpts(title="重庆各区二手房单价箱型图"), 
                 xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(interval=0)))
c.render("./html/重庆各区二手房单价箱型图.html")
c.render_notebook()

## 7. 重庆二手房房屋面积

In [9]:
new_df = pd.DataFrame(df['house_area'])
area_df = new_df.apply(lambda x : pd.cut(x, list(range(0, 530, 20)), labels=[str(n - 20) + '-' + str(n) for n in range(20, 530, 20)]))
series = area_df['house_area'].value_counts()
series.sort_index(ascending=True, inplace=True)
area_list = series.index.tolist()    # 房屋面积列表
count_list = series.values.tolist()    # 该面积数量列表


from pyecharts.charts import Line

c = Bar(init_opts=opts.InitOpts(theme=ThemeType.PURPLE_PASSION))
c.add_xaxis(area_list)
c.add_yaxis("重庆市", count_list, yaxis_index=0)
c.set_global_opts(title_opts=opts.TitleOpts(title="房屋面积与数量关系柱状图", subtitle=""),
                xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(interval=0, rotate=-60)))
c.render("./html/重庆二手房房屋面积.html")
c.render_notebook()

TypeError: '<' not supported between instances of 'int' and 'str'

In [33]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
f, [ax1,ax2] = plt.subplots(1, 2, figsize=(16, 6))

# 房屋面积
sns.histplot(df['house_area'], ax=ax1, color='r', kde=True, stat='density')
ax1.set_xlabel('面积')

# 房屋面积和价格的关系
sns.regplot(x='house_area', y='total_price', data=df, ax=ax2)
ax2.set_xlabel('面积')
ax2.set_ylabel('总价')

plt.show()

C:\ProgramData\miniconda3\lib\site-packages\IPython\core\pylabtools.py:152: UserWarning: Glyph 38754 (\N{CJK UNIFIED IDEOGRAPH-9762}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
C:\ProgramData\miniconda3\lib\site-packages\IPython\core\pylabtools.py:152: UserWarning: Glyph 31215 (\N{CJK UNIFIED IDEOGRAPH-79EF}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
C:\ProgramData\miniconda3\lib\site-packages\IPython\core\pylabtools.py:152: UserWarning: Glyph 24635 (\N{CJK UNIFIED IDEOGRAPH-603B}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
C:\ProgramData\miniconda3\lib\site-packages\IPython\core\pylabtools.py:152: UserWarning: Glyph 20215 (\N{CJK UNIFIED IDEOGRAPH-4EF7}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)


## 8. 重庆二手房各户型横向条形图

In [10]:
series = df['house_type'].value_counts()
series.sort_index(ascending=False, inplace=True)
house_type_list = series.index.tolist()
count_list = series.values.tolist()

c = Bar(init_opts=opts.InitOpts(theme=ThemeType.PURPLE_PASSION))
c.add_xaxis(house_type_list)
c.add_yaxis("重庆市", count_list)
c.reversal_axis()
c.set_series_opts(label_opts=opts.LabelOpts(position="right"))
c.set_global_opts(title_opts=opts.TitleOpts(title="重庆二手房各户型横向条形图"),
                datazoom_opts=[opts.DataZoomOpts(yaxis_index=0, type_="slider", orient="vertical")],)
c.render("./html/重庆二手房各户型横向条形图.html")
c.render_notebook()

## 9. 重庆二手房房屋装修饼状图

In [11]:
decoration_list = df['decoration'].value_counts().index.tolist()
count_list = df['decoration'].value_counts().values.tolist()

from pyecharts.charts import Pie

c = Pie(init_opts=opts.InitOpts(theme=ThemeType.PURPLE_PASSION))
c.add(series_name="房屋装修",
        data_pair=[list(z) for z in zip(decoration_list, count_list)],
        rosetype="radius",
        radius="55%",
        center=["50%", "50%"],
        label_opts=opts.LabelOpts(is_show=False, position="center"))
c.set_global_opts(title_opts=opts.TitleOpts(
                  title="重庆二手房房屋装修饼状图",
                  pos_left="center",
                  pos_top="20",
                  title_textstyle_opts=opts.TextStyleOpts(color="#fff")),
                  legend_opts=opts.LegendOpts(is_show=False))
c.set_series_opts(tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"),
                label_opts=opts.LabelOpts(color="rgba(255, 255, 255, 255)"))
c.render("./html/重庆二手房房屋装修饼状图.html")
c.render_notebook()

## 10. 重庆二手房有无电梯与房价关系图

In [12]:
df['elevator'].value_counts()
true_elevator_count = []    # 各区有电梯数量占比
false_elevator_count = []    # 各区无电梯数量占比
true_elevator_price = []    # 各区有电梯单价中位数
false_elevator_price = []    # 各区无电梯单价中位数
elevator_df = df.loc[df['elevator'] != '暂无数据']
for region in region_list:
    detail_df = elevator_df.loc[df['region'] == region]
    count_list = detail_df['elevator'].value_counts().values.tolist()
    ratio_list = [round(count / sum(count_list), 2) for count in count_list]
    true_elevator_count.append(ratio_list[0])
    false_elevator_count.append(ratio_list[1])
    true_elevator_price.append(detail_df.loc[detail_df['elevator'] == '有', 'unit_price'].median())
    false_elevator_price.append(detail_df.loc[detail_df['elevator'] == '无', 'unit_price'].median())
    

# 绘制关系图
from pyecharts.charts import Grid, Line

bar = (
    Bar()
    .add_xaxis(region_list)
    .add_yaxis(
        "有电梯",
        true_elevator_count,
        yaxis_index=1,
    )
    .add_yaxis(
        "无电梯",
        false_elevator_count,
        yaxis_index=1,
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            type_="value",
            name="数量占比",
            min_=0,
            max_=3,
            position="left",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="#d14a61")
            ),
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=1)
            ),
        )
    )
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            name="单价",
            min_=0,
            max_=30000,
            position="right",
            offset=0,
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="#5793f3")
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value} 元/m²"),
        ),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(interval=0)),
        title_opts=opts.TitleOpts(title="重庆二手房有无电梯与房价关系图"),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
    )
)

line = (
    Line()
    .add_xaxis(region_list)
    .add_yaxis(
        "有电梯",
        true_elevator_price,
        yaxis_index=0,
        label_opts=opts.LabelOpts(is_show=True),
    )
    .add_yaxis(
        "无电梯",
        false_elevator_price,
        yaxis_index=0,
        label_opts=opts.LabelOpts(is_show=True),
    )
)

bar.overlap(line)
grid = Grid(init_opts=opts.InitOpts(theme=ThemeType.PURPLE_PASSION))
grid.add(bar, opts.GridOpts(pos_left="5%", pos_right="20%"), is_control_axis_index=True)
grid.render('./html/重庆二手房有无电梯与房价关系图1.html')
grid.render_notebook()

In [20]:
bar = (
    Bar()
    .add_xaxis(region_list)
    .add_yaxis("有电梯", true_elevator_count)
    .add_yaxis("无电梯", false_elevator_count)
    .set_global_opts(title_opts=opts.TitleOpts(title="重庆二手房有无电梯数量占比柱状图"))
)
line = (
    Line()
    .add_xaxis(region_list)
    .add_yaxis("有电梯", true_elevator_price)
    .add_yaxis("无电梯", false_elevator_price)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="重庆二手房有无电梯与房价关系折线图", pos_top="48%"),
        legend_opts=opts.LegendOpts(pos_top="48%"),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(formatter="{value} 元/m²"),
        ),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(interval=0)),
    )
)

grid = (
    Grid(init_opts=opts.InitOpts(theme=ThemeType.PURPLE_PASSION))
    .add(bar, grid_opts=opts.GridOpts(pos_bottom="60%"))
    .add(line, grid_opts=opts.GridOpts(pos_top="60%"))

)
grid.render("./html/重庆二手房有无电梯与房价关系图2.html")
grid.render_notebook()

## 11. 重庆热门二手房关注人数关系漏斗图

In [21]:
from collections import Counter

# 只统计关注人数超过三人的热门二手房
detail_df = df.loc[df['follower_numbers'] > 3]
label_list = []
for house_label in detail_df['building_type'].values.tolist():
    label_list += house_label.split(',')
label_and_count = Counter(label_list)
label_and_count = label_and_count.most_common()

from pyecharts.charts import Funnel

c = Funnel(init_opts=opts.InitOpts(theme=ThemeType.PURPLE_PASSION))
c.add("商品", [list(z) for z in label_and_count])
c.set_global_opts(title_opts=opts.TitleOpts(title="热门标签漏斗图"))
c.render("./html/重庆热门二手房标签漏斗图.html")
c.render_notebook()

## 12. 重庆热门二手房标题关键词

In [22]:
def load_stopwords(read_path):
    '''
    读取文件每行内容并保存到列表中
    :param read_path: 待读取文件的路径
    :return: 保存文件每行信息的列表
    '''
    result = []
    with open(read_path, "r", encoding='utf-8') as f:
        for line in f.readlines():
            line = line.strip('\n')  # 去掉列表中每一个元素的换行符
            result.append(line)
    return result

# 加载中文停用词
stopwords = load_stopwords('wordcloud_stopwords.txt')

In [13]:
import jieba

# 添加自定义词典
jieba.load_userdict("自定义词典.txt")

token_list = []
# 对标题内容进行分词，并将分词结果保存在列表中
for title in detail_df['title']:
    tokens = jieba.lcut(title, cut_all=False)
    token_list += [token for token in tokens if token not in stopwords]
len(token_list)

ModuleNotFoundError: No module named 'jieba'

In [14]:
from pyecharts.charts import WordCloud
from collections import Counter

token_count_list = Counter(token_list).most_common(100)
new_token_list = []
for token, count in token_count_list:
    new_token_list.append((token, str(count)))

c = WordCloud()   
c.add(series_name="热词", data_pair=new_token_list, word_size_range=[20, 200])
c.set_global_opts(
    title_opts=opts.TitleOpts(
        title="重庆热门二手房标题关键词", title_textstyle_opts=opts.TextStyleOpts(font_size=23)
    ),
    tooltip_opts=opts.TooltipOpts(is_show=True),
)
c.render("./html/重庆热门二手房标题关键词.html")
c.render_notebook()

NameError: name 'token_list' is not defined